In [1]:
###Import required packages
from models import loss_biological, fa_layerednegativemetabolicloop, fa_negativegeneloop, fa_negativemetabolicloop, fa_openloop, fa_openloopintermediate
import numpy as np
from hyperopt import hp, fmin, tpe
from scikits.odes.ode import ode
import pandas as pd

In [2]:
###Specify if data is to be saved to CSV
save_data = False

In [3]:
###Search space definition
space = hp.choice('architecture',
    [('Open Loop', hp.uniform('r_lac_ol', 10E-11, 10E-8)),
        ('Open Loop Intermediate', [hp.uniform('r_lac_oli', 10E-11, 10E-8), hp.uniform('r_bad_oli', 10E-11, 10E-8)]),
        ('Negative Gene Loop', [hp.uniform('r_tl_ngl', 10E-11, 10E-8),hp.uniform('r_tl_tetR_ngl', 10E-11, 10E-8)]),
        ('Negative Metabolic Loop', [hp.uniform('r_fl_prime_nml', 10E-11, 10E-8), hp.uniform('ki_nml', 0., 0.12)]),
        ('Layered Negative Metabolic Loop', [hp.uniform('r_tl_lnml', 10E-11, 10E-8), hp.uniform('r_ar2_lnml', 10E-11, 10E-8),])
        ])

###Objective function
def run_opt(max_iters):
    losses = []
    params = []
    circuits = []

    def objective(args):
        architecture, param_values = args
        #Integration conditions
        t = np.linspace(0, 5E4, 200)

        #Select architecture and generate function
        if architecture == 'Open Loop':
            ode_function = fa_openloop
            y0 = np.array([0., 0., 0., 0.])
        elif architecture == 'Open Loop Intermediate':
            ode_function = fa_openloopintermediate
            y0 = np.array([0., 0., 0., 0., 0.])
        elif architecture == 'Negative Gene Loop':
            ode_function = fa_negativegeneloop
            y0 = np.array([0., 0., 0., 0., 0.])
        elif architecture == 'Negative Metabolic Loop':
            ode_function = fa_negativemetabolicloop
            y0 = np.array([0., 0., 0., 0.])
        else:
            ode_function = fa_layerednegativemetabolicloop
            y0 = np.array([0., 0., 0., 0., 0.])
        
        extra_options = {'old_api': False, 'user_data': param_values}
        ode_solver = ode('cvode', ode_function, **extra_options)
        solution = ode_solver.solve(t, y0)
        j1 = solution.values.y[-1][-1]
        j2 = solution.values.y[-1][-2]
        j1, j2, loss = loss_biological(j1, j2, alpha1=10E3, alpha2=10E-2)

        losses.append(loss)
        params.append(param_values)
        circuits.append(architecture)
        return loss

    #Run hyperopt call
    best = fmin(objective, space, algo=tpe.suggest, max_evals=max_iters)
    
    #Create trajectory data frame
    r_lac_ols, r_lac_olis, r_bad_olis, r_tl_ngls, r_tl_tetR_ngls, r_fl_prime_nmls, ki_nmls, r_tl_lnmls, r_ar2_lnmls = [[], [], [], [], [], [], [], [], []]
    for i in range(max_iters):
        r_lac_ol, r_lac_oli, r_bad_oli, r_tl_ngl, r_tl_tetR_ngl, r_fl_prime_nml, ki_nml, r_tl_lnml, r_ar2_lnml = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        arch = circuits[i]
        if arch == 'Open Loop':
            r_lac_ol = params[i]
        elif arch == 'Open Loop Intermediate':
            r_lac_oli  = params[i][0]
            r_bad_oli = params[i][1]
        elif arch == 'Negative Gene Loop':
            r_tl_ngl = params[i][0]
            r_tl_tetR_ngl = params[i][1]
        elif arch == 'Negative Metabolic Loop':
            r_fl_prime_nml = params[i][0]
            ki_nml = params[i][1]
        else:
            r_tl_lnml = params[i][0]
            r_ar2_lnml = params[i][1]
        
        r_lac_ols.append(r_lac_ol)
        r_lac_olis.append(r_lac_oli)
        r_bad_olis.append(r_bad_oli)
        r_tl_ngls.append(r_tl_ngl)
        r_tl_tetR_ngls.append(r_tl_tetR_ngl)
        r_fl_prime_nmls.append(r_fl_prime_nml)
        ki_nmls.append(ki_nml)
        r_tl_lnmls.append(r_tl_lnml)
        r_ar2_lnmls.append(r_ar2_lnml)
        
    landscape = pd.DataFrame({'circuit':circuits, 'loss': losses, 'r_lac_ol':r_lac_ols, 'r_lac_oli':r_lac_olis, 'r_bad_oli':r_bad_olis,
            'r_tl_ngl':r_tl_ngls, 'r_tl_tetR_ngl':r_tl_tetR_ngls, 'r_fl_prime_nml':r_fl_prime_nmls,
            'ki_nml':ki_nmls, 'r_tl_lnml':r_tl_lnmls, 'r_ar2_lnml':r_ar2_lnmls})    

    best_loss = 1E5
    best_circuit = 'Initial'
    best_losses = []
    best_losses_circuits = []
    for i in range(len(landscape)):
        if landscape.loss[i] < best_loss:
            best_loss = landscape.loss[i]
            best_circuit = landscape.circuit[i]
        best_losses.append(best_loss)
        best_losses_circuits.append(best_circuit)
    landscape['best_losses'] = best_losses
    landscape['best_loss_circuit'] = best_losses_circuits
    
    return landscape, best

In [ ]:
###Run sample optimization
max_iters = 1000
landscape, best = run_opt(max_iters)
landscape = landscape.reset_index()
if save_data: landscape.to_csv('../data/fatty_acid_sample_run.csv')

In [ ]:
###Run N=100 times to check best architectures
num_replicates = 100
max_iters = 500

space = hp.choice('architecture',
    [('Open Loop', hp.uniform('r_lac_ol', 10E-11, 10E-8)),
        ('Open Loop Intermediate', [hp.uniform('r_lac_oli', 10E-11, 10E-8), hp.uniform('r_bad_oli', 10E-11, 10E-8)]),
        ('Negative Gene Loop', [hp.uniform('r_tl_ngl', 10E-11, 10E-8),hp.uniform('r_tl_tetR_ngl', 10E-11, 10E-8)]),
        ('Negative Metabolic Loop', [hp.uniform('r_fl_prime_nml', 10E-11, 10E-8), hp.uniform('ki_nml', 0., 0.12)]),
        ('Layered Negative Metabolic Loop', [hp.uniform('r_tl_lnml', 10E-11, 10E-8), hp.uniform('r_ar2_lnml', 10E-11, 10E-8),])
        ])

total_landscape = pd.DataFrame()
for i in range(num_replicates):
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    landscape['iteration'] = landscape['index']
    landscape['replicate'] = i+1
    total_landscape = pd.concat([total_landscape, landscape])
if save_data: total_landscape.to_csv('../data/fatty_acid_background.csv')

In [ ]:
###Run N=100 times and compute parameter robustness - takes about 30 min
num_replicates = 100
max_iters = 500

space = hp.choice('architecture',
    [('Open Loop', hp.uniform('r_lac_ol', 10E-11, 10E-8))])
ol_landscape = pd.DataFrame()
for i in range(num_replicates):
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    landscape['iteration'] = landscape['index']
    landscape['replicate'] = i+1
    ol_landscape = pd.concat([ol_landscape, landscape])

space = hp.choice('architecture',
    [('Open Loop Intermediate', [hp.uniform('r_lac_oli', 10E-11, 10E-8), hp.uniform('r_bad_oli', 10E-11, 10E-8)])])
oli_landscape = pd.DataFrame()
for i in range(num_replicates):
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    landscape['iteration'] = landscape['index']
    landscape['replicate'] = i+1
    oli_landscape = pd.concat([oli_landscape, landscape])

space = hp.choice('architecture',
    [('Negative Gene Loop', [hp.uniform('r_tl_ngl', 10E-11, 10E-8),hp.uniform('r_tl_tetR_ngl', 10E-11, 10E-8)])])
ngl_landscape = pd.DataFrame()
for i in range(num_replicates):
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    landscape['iteration'] = landscape['index']
    landscape['replicate'] = i+1
    ngl_landscape = pd.concat([ngl_landscape, landscape])

space = hp.choice('architecture',
    [('Negative Metabolic Loop', [hp.uniform('r_fl_prime_nml', 10E-11, 10E-8), hp.uniform('ki_nml', 0., 0.12)])])
nml_landscape = pd.DataFrame()
for i in range(num_replicates):
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    landscape['iteration'] = landscape['index']
    landscape['replicate'] = i+1
    nml_landscape = pd.concat([nml_landscape, landscape])

space = hp.choice('architecture',
    [('Layered Negative Metabolic Loop', [hp.uniform('r_tl_lnml', 10E-11, 10E-8), hp.uniform('r_ar2_lnml', 10E-11, 10E-8),])])
lnml_landscape = pd.DataFrame()
for i in range(num_replicates):
    landscape, best = run_opt(max_iters)
    landscape = landscape.reset_index()
    landscape['iteration'] = landscape['index']
    landscape['replicate'] = i+1
    lnml_landscape = pd.concat([lnml_landscape, landscape])

total_landscape = pd.concat([ol_landscape, oli_landscape, ngl_landscape, nml_landscape, lnml_landscape])
if save_data: total_landscape.to_csv('../data/fatty_acid_singlearch.csv')

In [ ]:
###Run sweep of landscape
def objective(args):
        architecture, param_values = args
        #Integration conditions
        t = np.linspace(0, 5E4, 200)

        #Select architecture and generate function
        if architecture == 'Open Loop':
            ode_function = fa_openloop
            y0 = np.array([0., 0., 0., 0.])
        elif architecture == 'Open Loop Intermediate':
            ode_function = fa_openloopintermediate
            y0 = np.array([0., 0., 0., 0., 0.])
        elif architecture == 'Negative Gene Loop':
            ode_function = fa_negativegeneloop
            y0 = np.array([0., 0., 0., 0., 0.])
        elif architecture == 'Negative Metabolic Loop':
            ode_function = fa_negativemetabolicloop
            y0 = np.array([0., 0., 0., 0.])
        else:
            ode_function = fa_layerednegativemetabolicloop
            y0 = np.array([0., 0., 0., 0., 0.])
        
        extra_options = {'old_api': False, 'user_data': param_values}
        ode_solver = ode('cvode', ode_function, **extra_options)
        solution = ode_solver.solve(t, y0)
        j1 = solution.values.y[-1][-1]
        j2 = solution.values.y[-1][-2]
        j1, j2, loss = loss_biological(j1, j2, alpha1=10E3, alpha2=10E-2)

        return loss

landscape_grid = pd.DataFrame()
architecture = 'Open Loop Intermediate'
param1s = np.linspace(10.5E-11, 10.5E-8, 10)
param2s = np.linspace(10.5E-11, 10.5E-8, 10)
for p1 in param1s:
    for p2 in param2s:
        landscape = pd.DataFrame({'index':[0], 'architecture': architecture, 'param1': p1, 'param2':p2, 'loss':objective([architecture, [p1, p2]])})
        landscape_grid = pd.concat([landscape_grid, landscape])
    
architecture = 'Negative Gene Loop'
param1s = np.linspace(10E-11, 10E-8, 10)
param2s = np.linspace(10E-11, 10E-8, 10)
for p1 in param1s:
    for p2 in param2s:
        landscape = pd.DataFrame({'index':[0], 'architecture': architecture, 'param1': p1, 'param2':p2, 'loss':objective([architecture, [p1, p2]])})
        landscape_grid = pd.concat([landscape_grid, landscape])

architecture = 'Negative Metabolic Loop'
param1s = np.linspace(10E-11, 10E-8, 10)
param2s = np.linspace(0.000000000001, 0.12, 10)
for p1 in param1s:
    for p2 in param2s:
        landscape = pd.DataFrame({'index':[0], 'architecture': architecture, 'param1': p1, 'param2':p2, 'loss':objective([architecture, [p1, p2]])})
        landscape_grid = pd.concat([landscape_grid, landscape])

architecture = 'Layered Negative Metabolic Loop'
param1s = np.linspace(10E-11, 10E-8, 10)
param2s = np.linspace(10E-11, 10E-8, 10)
for p1 in param1s:
    for p2 in param2s:
        landscape = pd.DataFrame({'index':[0], 'architecture': architecture, 'param1': p1, 'param2':p2, 'loss':objective([architecture, [p1, p2]])})
        landscape_grid = pd.concat([landscape_grid, landscape])

if save_data: landscape_grid.to_csv('fatty_acid_landscapes_grid_search.csv')